# 🩸 Blood Demand Forecast & Alert System
This notebook simulates hospital blood demand data, builds a machine learning model, and creates a Streamlit app with real-time user input and alert logic.

In [4]:
import pandas as pd
import numpy as np

np.random.seed(42)

hospitals = ['AKTH', 'MMSH', 'NAS']
blood_types = ['O+', 'A+', 'B-', 'AB+']
dates = pd.date_range(start='2021-01-01', end='2023-12-31')

data = []
for date in dates:
    for h in hospitals:
        for bt in blood_types:
            accident_cases = np.random.poisson(5)
            units_needed = int(np.clip(accident_cases * np.random.uniform(0.8, 1.5), 1, 25))
            units_available = units_needed + np.random.randint(-5, 5)
            data.append({
                'date': date,
                'hospital': h,
                'blood_type': bt,
                'accident_cases': accident_cases,
                'blood_units_needed': units_needed,
                'blood_units_available': max(units_available, 0),
                'is_weekend': int(date.weekday() >= 5),
                'is_public_holiday': int(np.random.rand() < 0.05)
            })

df = pd.DataFrame(data)
df.to_csv("kano_blood_demand_2021_2023.csv", index=False)
df.head()


,date,hospital,blood_type,accident_cases,blood_units_needed,blood_units_available,is_weekend,is_public_holiday
0,2021-01-01,AKTH,O+,5,4,6,0,0
1,2021-01-01,AKTH,A+,2,2,2,0,1
2,2021-01-01,AKTH,B-,3,2,6,0,1
3,2021-01-01,AKTH,AB+,4,5,8,0,1
4,2021-01-01,MMSH,O+,3,4,0,0,0


In [8]:
%%writefile app.py
import streamlit as st
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
import altair as alt

st.set_page_config(page_title="Blood Demand Forecast", layout="centered")
st.title("🩸 Real-Time Blood Demand Forecast & Alert System (Kano)")

@st.cache_data
def load_data():
    return pd.read_csv("kano_blood_demand_2021_2023.csv")

df = load_data()
st.success("✅ Data Loaded Successfully")

hospital_encoder = LabelEncoder().fit(df['hospital'])
blood_encoder = LabelEncoder().fit(df['blood_type'])

df['hospital_encoded'] = hospital_encoder.transform(df['hospital'])
df['blood_type_encoded'] = blood_encoder.transform(df['blood_type'])

features = ['hospital_encoded', 'blood_type_encoded', 'accident_cases', 'is_weekend', 'is_public_holiday']
X = df[features]
y = df['blood_units_needed']

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X, y)

# 📤 Upload Inventory
st.subheader("📁 Upload Inventory (CSV)")
uploaded_file = st.file_uploader("Upload a CSV file with columns: hospital, blood_type, stock", type="csv")
if uploaded_file is not None:
    user_inventory = pd.read_csv(uploaded_file)
    inventory = {(row['hospital'], row['blood_type']): row['stock'] for _, row in user_inventory.iterrows()}
    st.success("✅ Inventory updated from uploaded file!")
    st.dataframe(user_inventory)
else:
    st.info("Using default inventory values.")
    inventory = {
        ('AKTH', 'O+'): 12,
        ('AKTH', 'A+'): 8,
        ('MMSH', 'O+'): 10,
        ('MMSH', 'B-'): 5,
        ('NAS', 'AB+'): 6
    }

# 📥 User Input
st.subheader("📥 Enter Forecast Parameters")
hospital = st.selectbox("Select Hospital", hospital_encoder.classes_)
blood_type = st.selectbox("Select Blood Type", blood_encoder.classes_)
accidents = st.slider("Estimated Accident Cases", 0, 50, 10)
is_weekend = st.checkbox("Is Weekend?", value=False)
is_holiday = st.checkbox("Is Public Holiday?", value=False)

if st.button("🔮 Predict Demand"):
    hosp_encoded = hospital_encoder.transform([hospital])[0]
    blood_encoded = blood_encoder.transform([blood_type])[0]

    input_df = pd.DataFrame([{
        'hospital_encoded': hosp_encoded,
        'blood_type_encoded': blood_encoded,
        'accident_cases': accidents,
        'is_weekend': int(is_weekend),
        'is_public_holiday': int(is_holiday)
    }])

    predicted = model.predict(input_df)[0]
    st.markdown(f"### 🔢 Predicted Demand: **{predicted:.1f} units**")

    stock = inventory.get((hospital, blood_type), 0)
    if predicted > stock:
        st.error(f"⚠️ ALERT: Predicted demand exceeds inventory (Stock: {stock})")
    else:
        st.success(f"✅ Safe: Inventory sufficient (Stock: {stock})")

# 📊 Weekly Chart
st.subheader("📈 Weekly Demand Trends by Hospital")
df['week'] = pd.to_datetime(df['date']).dt.to_period('W').astype(str)
weekly = df.groupby(['week', 'hospital'])['blood_units_needed'].sum().reset_index()

chart = alt.Chart(weekly).mark_line(point=True).encode(
    x='week:T',
    y='blood_units_needed:Q',
    color='hospital:N',
    tooltip=['week', 'hospital', 'blood_units_needed']
).properties(width=700, height=400)

st.altair_chart(chart, use_container_width=True)


Overwriting app.py


In [9]:
import subprocess
import sys

subprocess.Popen([sys.executable, "-m", "streamlit", "run", "app.py"])

<Popen: returncode: None args: ['c:\\Users\\USER\\AppData\\Local\\Programs\\...>